# Process Checks

In [18]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Tickerplant code

In [19]:
from qpython.qconnection import QConnection as qcon
from qpython.qcollection import QDictionary as qdict
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import curses
%matplotlib inline

#set qcon variables
import csv
with open('credentials.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Credentials required are {", ".join(row)}')
            line_count += 1
        else:
            host=row[0]
            un=row[1]
            pswd=row[2]
            print(f'\t host:{row[0]} username: {row[1]} password: {row[2]}.')
            line_count += 1
    print(f'Processed {line_count} lines.')      
                  

Credentials required are host, username, password
	 host:localhost username: admin password: admin.
Processed 2 lines.


In [20]:
#run torq summary
torq_sum= ! "$(cd "$(dirname "${BASH_SOURCE[0]}")" && pwd)"/torq.sh summary

#convert array to numpy array
summary=np.asarray(torq_sum)

#split each element of array by | character
sum_list = [i.split('|') for i in summary]
df = pd.DataFrame(sum_list)

# take first row and use as header for df
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

#trim whitespace from headers
cols=[]
for i in df.columns:
    cols.append(str.strip(i))
df.columns=cols

#trim whitespace from all objects within dataframe
data = df.select_dtypes(['object'])
df[data.columns] = df.apply(lambda x: x.str.strip())

# function to extract the port number for each process - takes a string argument
def find_portno(process):
    process_info = df.loc[df['PROCESS'] == process]
    PORT = process_info['PORT'].astype(str).astype(int)
    return PORT

#function to decode bytes to strings
def byte_decode(table,cols):
    table[cols] = table[cols].applymap(lambda x: x.decode('utf-8'))

### Process Summary

Table showing process name, status, PID and Port number.
* Process status indicated by colours green (up) and red (down).
* Killtick, tpreplay1 and compression1 should usually have a down status indicated.

In [21]:
# set colour on down processes to red and up processes to green
def colour_down_red(col):
    color = 'red' if 'down' in col else 'green'
    return 'color: %s' % color
df.style.applymap(colour_down_red, subset=['STATUS'])

,TIME,PROCESS,STATUS,PID,PORT
1,10:09:40,discovery1,up,14779,1701
2,10:09:40,tickerplant1,up,14871,1700
3,10:09:40,rdb1,up,14961,1702
4,10:09:40,hdb1,up,15053,1703
5,10:09:40,hdb2,up,15145,1704
6,10:09:40,wdb1,up,15237,1705
7,10:09:40,sort1,up,15329,1706
8,10:09:40,gateway1,up,15421,1707
9,10:09:40,killtick,down,,None
10,10:09:41,monitor1,up,15513,1709


### Count of tables in Tickerplant 

Table to show the count in each table found in the Tickerplant 
* The counts in each of these tables should be 0, as the Tickerplant should not be storing any data.
* If the counts in any of these tables is not 0, this could indicate a slow subscriber.

In [22]:
#table to show tables in TP and the counts of each of them
##counts should all be zero
with qcon(host, port=find_portno('tickerplant1'), username=un, password=pswd,timeout=3.0) as q:
    tablecounts = q("enlist tables[]!count each value each tables[]", pandas=True)
tablecounts

,logmsg,quote,quote_iex,trade,trade_iex
0,0,0,0,0,0


### Tickerplant Log file size increasing

Checks if log messages in the log file of the tickerplant is increasing.
   * If log messages are increasing, the tickerplant is receiving data.
   * If log messages are not increasing, the tickerplant may not be recieving data. 


In [23]:
#log files are increasing over time
with qcon(host, port=find_portno('tickerplant1'), username=un, password=pswd,timeout=3.0) as q:
    log1=(q("hcount .u.L"))
    time.sleep(2)
    log2=(q("hcount .u.L"))
print ("Log file sizes are increasing: ", log1<log2)

Log file sizes are increasing:  True


### Process handles connected to Tickerplant

Table to show the handles of processes connected to the tickerplant and if there are any slow subscribers.
  *  A process may be a slow subscriber if there is a number value in the output queue.

In [24]:
# shows IPC handles with number of bytes waiting in their output queues and what processes are connected
##shows any slow subscribers 
with qcon(host, port=find_portno('tickerplant1'), username=un, password=pswd,timeout=3.0) as q:
    zW=q(".z.W[]", pandas=True)
    hprocesses=q("asc select w,u from .clients.clients", pandas=True)
    byte_decode(hprocesses, ['u'])
# assign columns with new names
keys=pd.DataFrame(zW.keys, columns=['handles'])
values=pd.DataFrame(zW.values, columns=['output queue'])
hprocesses=hprocesses.rename(columns={'u':'processes'})
# apply new names
zW2=keys.join(values)
# join zW2 and hprocesses
zW2=zW2.join(hprocesses.processes)
zW2.set_index('handles', inplace=True)
zW2

,output queue,processes
handles,,
6,NaN,rdb
7,NaN,wdb
8,NaN,feed
9,NaN,chainedtp
10,NaN,metrics
11,NaN,iexfeed
12,NaN,admin


## RDB code

In [25]:
with qcon(host, port=find_portno('rdb1'), username=un, password=pswd,timeout=3.0) as q:
    
    #Check tables in rdb are same as tables in tickerplant 
    tables = q('all 1_tables[] in ((exec w from .servers.SERVERS where proctype=`tickerplant)0)("tables[]")')
    #Check count of tables in rdb - data is being sent from the tickerplant 
    tptordb = q('enlist tables[]!count each value each tables[]', pandas=True)
    tabname=q('tables[](2)')
    #Check that data in table can be queried
    rdbtquery=q('5#select from tables[](2)', pandas=True)
    byte_decode(rdbtquery,['sym'])
    
    #Check that only data from today is present in the rdb tables
    onedatet = q('select Currentdate:all .z.d=distinct (`date$time) from tables[](2)', pandas=True)
    #ondedateq = q('select Currentdate:all .z.d=distinct (`date$time) from quote', pandas=True)

### RDB tables

Checks if the tables in the rdb are the same as the tables in the Tickerplant.

In [26]:
print ("RDB tables are same as Tickerplant tables : ", tables)

RDB tables are same as Tickerplant tables :  True


### RDB table counts

Checks to see if data is being sent from the Tickerplant to the RDB
* Counts for heartbeat and logmsg should be 0
* If counts for the other tables are 0, the RDB has been sent no data for today

In [27]:
tptordb.set_index(tptordb.columns.tolist())

,,,,,
heartbeat,logmsg,quote,quote_iex,trade,trade_iex
0,0,10365,0,1990,0


### RDB Query
Checks to see if tables in the RDB can be queried 

In [28]:
print ('Table to be queried:')
name = tabname.decode('UTF-8')
print (name)
rdbtquery

Table to be queried:
quote


,time,sym,bid,ask,bsize,asize,mode,ex
0,2019-11-01 10:06:59.333862,DOW,19.70,20.28,47,99,R,O
1,2019-11-01 10:06:59.333862,AMD,32.99,33.65,26,66,N,N
2,2019-11-01 10:06:59.333862,DELL,11.94,12.45,31,84,Y,N
3,2019-11-01 10:06:59.333862,DOW,19.08,20.67,22,64,I,O
4,2019-11-01 10:06:59.333862,AMD,33.00,33.82,92,48,L,N


### RDB Date Checks
Check to see if date in RDB tables is the current date

In [29]:
def color_true(val):
    color  ='pink' if val==False else 'green'
    return 'background-color: %s' %color
onedatet.style.applymap(color_true)

,Currentdate
0,True


### HDB Code

In [30]:
with qcon(host, port=find_portno('hdb1'), username=un, password=pswd,timeout=3.0) as q:
    
    #Check hdb table counts excl. eod_summary and eod_summary_iex
    hdbtablecount=q('enlist tables[]!count each value each tables[]',pandas=True)

    
    lastdayquery=q('5#select from tables[](2) where date=.z.d-1', pandas=True)
    byte_decode(lastdayquery,['sym'])

### HDB Table Counts
* Counts for heartbeat and logmsg should be 0
* If counts for other tables are 0, no data has been recieved for the day before

In [31]:
hdbtablecount.set_index(hdbtablecount.columns.tolist())

,,,,,
heartbeat,logmsg,quote,quote_iex,trade,trade_iex
0,0,13522644,0,2961211,0


### HDB Query
Check to see if HDB tables can be queried

#### HDB Trade table query 

In [32]:
print ('Table to be queried:')
print (name)
lastdayquery

Table to be queried:
quote


,date,time,sym,bid,ask,bsize,asize,mode,ex
0,NaT,NaT,,NaN,NaN,NaN,NaN,,
1,NaT,NaT,,NaN,NaN,NaN,NaN,,
2,NaT,NaT,,NaN,NaN,NaN,NaN,,
3,NaT,NaT,,NaN,NaN,NaN,NaN,,
4,NaT,NaT,,NaN,NaN,NaN,NaN,,


### Process Memory Usage

Table to show memory usage of each process

In [57]:
!ps -e -o %p,  -o %C, -o %mem -o ,%a | grep $KDBBASEPORT >procmem.csv #process status commmand for TorQ processes
df3=pd.read_csv("./procmem.csv") 
df3.columns=('PID','%CPU','%MEM','COMMAND')
df3.style.set_properties(subset=['COMMAND'], **{'width': '700px'})

,PID,%CPU,%MEM,COMMAND
0,14871,0.1,0.2,q /home/nwatterson/jupyter_torQ/deploy/TorQ/torq.q -stackid 1700 -proctype tickerplant -procname tickerplant1 -U /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/passwords/accesslist.txt -localtime 1 -g 0 -load /home/nwatterson/jupyter_torQ/deploy/TorQ/code/processes/tickerplant.q -schemafile /home/nwatterson/jupyter_torQ/deploy/TorQ/database -tplogdir /home/nwatterson/jupyter_torQ/deploy/TorQ/hdb -procfile /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/process.csv
1,14961,0.1,1.4,q /home/nwatterson/jupyter_torQ/deploy/TorQ/torq.q -stackid 1700 -proctype rdb -procname rdb1 -U /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/passwords/accesslist.txt -localtime 1 -g 1 -T 180 -load /home/nwatterson/jupyter_torQ/deploy/TorQ/code/processes/rdb.q -procfile /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/process.csv
2,15053,0,0.3,q /home/nwatterson/jupyter_torQ/deploy/TorQ/torq.q -stackid 1700 -proctype hdb -procname hdb1 -U /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/passwords/accesslist.txt -localtime 1 -g 1 -T 60 -w 4000 -load /home/nwatterson/jupyter_torQ/deploy/TorQ/hdb/database -procfile /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/process.csv
3,15145,0.1,0.3,q /home/nwatterson/jupyter_torQ/deploy/TorQ/torq.q -stackid 1700 -proctype hdb -procname hdb2 -U /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/passwords/accesslist.txt -localtime 1 -g 1 -T 60 -w 4000 -load /home/nwatterson/jupyter_torQ/deploy/TorQ/hdb/database -procfile /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/process.csv
4,15237,0.1,0.3,q /home/nwatterson/jupyter_torQ/deploy/TorQ/torq.q -stackid 1700 -proctype wdb -procname wdb1 -U /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/passwords/accesslist.txt -localtime 1 -g 1 -load /home/nwatterson/jupyter_torQ/deploy/TorQ/code/processes/wdb.q -procfile /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/process.csv
5,15329,0.1,0.3,q /home/nwatterson/jupyter_torQ/deploy/TorQ/torq.q -stackid 1700 -proctype sort -procname sort1 -U /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/passwords/accesslist.txt -localtime 1 -g 1 -load /home/nwatterson/jupyter_torQ/deploy/TorQ/code/processes/wdb.q -s -2 -procfile /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/process.csv
6,15421,0.1,0.3,q /home/nwatterson/jupyter_torQ/deploy/TorQ/torq.q -stackid 1700 -proctype gateway -procname gateway1 -U /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/passwords/accesslist.txt -localtime 1 -g 1 -w 4000 -load /home/nwatterson/jupyter_torQ/deploy/TorQ/code/processes/gateway.q -procfile /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/process.csv
7,15513,0.1,0.3,q /home/nwatterson/jupyter_torQ/deploy/TorQ/torq.q -stackid 1700 -proctype monitor -procname monitor1 -localtime 1 -g 0 -load /home/nwatterson/jupyter_torQ/deploy/TorQ/code/processes/monitor.q -procfile /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/process.csv
8,15605,0,0.3,q /home/nwatterson/jupyter_torQ/deploy/TorQ/torq.q -stackid 1700 -proctype housekeeping -procname housekeeping1 -U /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/passwords/accesslist.txt -localtime 1 -g 0 -load /home/nwatterson/jupyter_torQ/deploy/TorQ/code/processes/housekeeping.q -procfile /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/process.csv
9,15697,0.1,0.3,q /home/nwatterson/jupyter_torQ/deploy/TorQ/torq.q -stackid 1700 -proctype reporter -procname reporter1 -U /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/passwords/accesslist.txt -localtime 1 -g 0 -load /home/nwatterson/jupyter_torQ/deploy/TorQ/code/processes/reporter.q -procfile /home/nwatterson/jupyter_torQ/deploy/TorQ/appconfig/process.csv
